In [1]:
import numpy as np
import pandas as pd
import altair as alt
import datetime
import requests

Ok so we gotta read the stuff in from my google sheet where I track time.

In [ ]:
sheetUrl = 'https://docs.google.com/spreadsheets/d/1Qq13N2dgpU8TWwLLJKsmauKQM1Damvykkn2nOnFQ56A/edit#gid=580980480'
csvFromSheet = '/'.join(sheetUrl.split('/')[:-1] + ['export?format=csv'])
df = pd.read_csv(csvFromSheet, parse_dates=['start', 'end'])
rawData = df[df['start'] != df['end']].sort_values('start')

See? Run the cell below using ctrl/cmd + enter

In [19]:
rawData

,activity,start,end,hours
0,night reading,2021-03-03 18:28:16,2021-03-03 18:34:55,0.110780
1,wake up,2021-03-04 02:28:24,2021-03-04 04:09:47,1.689601
2,school,2021-03-04 04:09:47,2021-03-04 05:10:50,1.017760
3,food,2021-03-04 05:10:50,2021-03-04 06:07:20,0.941574
4,school,2021-03-04 06:07:20,2021-03-04 07:08:00,1.011135
...,...,...,...,...
441,school,2021-03-26 08:59:39,2021-03-26 10:05:49,1.102786
442,pomodoroBreak,2021-03-26 10:05:49,2021-03-26 10:15:35,0.162821
443,school,2021-03-26 10:15:35,2021-03-26 11:12:05,0.941549
444,lift,2021-03-26 11:15:35,2021-03-26 12:15:35,NaN


Basically take rawData and make graphs in Altair. The graph below was hard to make... maybe try something simpler to start.

In [21]:

# Insert NA between things
fillAfter = rawData.loc[rawData['start'].shift(-1) != rawData['end'], 'end']
NAsDict = {'activity': ["N/A"]*len(fillAfter), "start": fillAfter, "end": [None]*len(fillAfter)}
NAs = pd.DataFrame(NAsDict)[:-1]

# These are for the beginning and end
beginningEndDict = {
    'activity': ["N/A"]*2, 
    "start": [min(rawData['start']).floor('D'), max(rawData['end'])], 
    "end": [min(rawData['start']), max(rawData['end']).ceil('D')]
}
beginningEnd = pd.DataFrame(beginningEndDict)

# This should come after everything is partitioned
midnightSeries = pd.date_range(min(rawData['start']).ceil('D'), max(rawData['end']).floor('D'))
midnightsDict = {'activity': [None]*len(midnightSeries), 'start': midnightSeries, 'end': [None]*len(midnightSeries)}
midnights = pd.DataFrame(midnightsDict)

# combine the NA with the non-NA
all = pd.concat([rawData, NAs, beginningEnd, midnights], ignore_index=True)
all = all.sort_values('start')
# Fill NA for added rows with correct values
all['end'] = all['end'].fillna(all['start'].shift(-1))
all['activity'] = all['activity'].fillna(method='ffill')
# For days crossing over, truncate at midnight
all.loc[all['start'].dt.day != all['end'].dt.day, 'end'] = all['start'].dt.ceil('D')
all.loc[all['start'] == all['end'], 'end'] = all['start'].shift(-1)
all['Hour'] = (all['end'] - all['start']).dt.total_seconds()/3600
all['day'] = all['start'].dt.floor('D')

selection = alt.selection_multi(fields=['activity'])
# multi = alt.selection_multi()
alt.Chart(all).mark_bar().encode(
    x = alt.X('monthdate(start):O', axis=alt.Axis(title='Date')),
    y=alt.Y('Hour', axis=alt.Axis(title="Time"), scale=alt.Scale(domain=(0, 24), clamp=True)), 
    order=alt.Order('start'), 
    color=alt.condition(alt.datum.activity == 'N/A', alt.value('white'), alt.Color('activity', legend=None)),
    opacity=alt.condition(selection, alt.value(1), alt.value(.2)),
    tooltip=[
        'activity:N', 
        alt.Tooltip('Hour:Q', format='.2f', title='hours'), 
        alt.Tooltip('start:T', format='%-I:%M %p', title='started'),
        alt.Tooltip('end:T', format='%-I:%M %p', title='ended')]
).add_selection(
    selection
)

alt.Chart(...)